# Libraries

In [43]:
import numpy as np
import time
import matplotlib.pyplot as plt
import cv2
import yaml

In [78]:
tdmclient.notebook.stop()

<coroutine object stop at 0x000001D1933A0450>

# Thymio

In [164]:
import time
# Import tdmclient Notebook environment:
import tdmclient.notebook

await tdmclient.notebook.start()

@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):
    global motor_left_target, motor_right_target
    
    # Printing the speeds if requested
    if verbose:
        print("\t\t Setting speed : ", l_speed, r_speed)
    motor_left_target = l_speed
    motor_right_target = r_speed
    return motor_left_speed,motor_right_speed
    

l,r =motors(0, 0, verbose=True) #test with lower speed value
print(l,r)

		 Setting speed :  0 0
0 0


In [13]:

motor_left_target=20
motor_right_target=20


In [93]:
@tdmclient.notebook.sync_var
def speeds():
    global motor_left_speed,motor_right_speed
    return motor_left_speed ,motor_right_speed

In [185]:
0.0975/2

0.04875

# Extended Kalman Filter

In [347]:

class KalmanFilter:
    def __init__(self, robot_length=0.011, speed_variance= 0.048755 ,conversion_factor=0.0387096):
        self.X = np.array([0, 0,0], dtype=float)  
        self.U = np.array([0, 0], dtype=float)  
        self.P = np.eye(3)  
        self.Z = np.array([0, 0, 0], dtype=float)  # Measurement
        self.robot_length = robot_length
        self.speed_variance = speed_variance
        self.conversion_factor=conversion_factor
        self.theta=0
        
    def prediction_step(self, period):
        
        if period < 0.01:
            dt = 0.01 * self.conversion_factor/2
            time.sleep(0.01-period)
        else:
            dt = period * self.conversion_factor/2
        
        A = np.eye(3)  
        theta = self.X[2]  
        
        B = np.array([[0.5 * dt * np.cos(theta), 0.5 * dt * np.cos(theta)],
                      [-0.5 * dt * np.sin(theta), -0.5 * dt * np.sin(theta)],
                      [dt / self.robot_length, -dt / self.robot_length]])

        X_predicted = np.dot(A, self.X) + np.dot(B, self.U)

        Q = np.eye(2) * self.speed_variance
        P_predicted = self.P + np.dot(np.dot(B, Q), B.T)

        self.X = X_predicted
        self.P = P_predicted
        return self.X, self.P

    def update_step(self, Z, aruco_detected):
        
        # Measurement noise covariance
        R = np.diag([0.0006, 0.0003, 0.0002]) 
        
        # Measurement matrix
        H = np.eye(3) if aruco_detected else np.zeros((3, 3))  

        # Innovation (difference between measurement and prediction)
        I = Z - np.dot(H, self.X)

        # Innovation covariance
        S = np.dot(np.dot(H, self.P), H.T) + R

        # Kalman gain
        K = np.dot(np.dot(self.P, H.T), np.linalg.inv(S))

        # State update
        self.X = self.X + np.dot(K, I)

        # Covariance update
        self.P = np.dot(np.eye(3) - np.dot(K, H), self.P)

        return self.X, self.P

    def kalman_filter(self, aruco_detected, U, Z, period):
        self.U = U  # Update control input

        # Prediction step
        X_predicted, P_predicted = self.prediction_step(period)

        # Update step 
        if aruco_detected:
            self.X, self.P = self.update_step(Z, aruco_detected)

        return self.X, self.P, X_predicted


# Aruco EKF

In [47]:

class Image_EKF:
    def __init__(self):
        pass
    
    @staticmethod
    def initialize_camera_and_calibration(calibration_file = 'calibration.yaml', camera_index=1, aruco_dict_type=cv2.aruco.DICT_6X6_1000):
        
        with open(calibration_file, 'r') as f:
            calibration_data = yaml.safe_load(f)
    
        mtx = np.array(calibration_data['camera_matrix'])
        dist = np.array(calibration_data['dist_coeff'])
    
        # Load the ArUco dictionary and parameters
        aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)
        aruco_params = cv2.aruco.DetectorParameters()
    
        # Initialize the camera
        camera = cv2.VideoCapture(camera_index)
        if not camera.isOpened():
            raise RuntimeError("Error: Could not open the camera.")
    
        # Read a frame to determine the image size
        ret, img = camera.read()
        if not ret:
            raise RuntimeError("Error: Could not read from the camera.")
    
        h, w = img.shape[:2]
    
        # Get the optimal camera matrix
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
    
        return camera, aruco_dict, aruco_params, newcameramtx, dist, mtx 
    
    @staticmethod    
    def detect_marker_with_id(frame, aruco_dict, aruco_params, target_id=10):
        robot_detected=False
        # Detect markers in the frame
        corners, ids, rejected_img_points = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=aruco_params)
    
        if ids is not None:
            for i, marker_id in enumerate(ids.flatten()):
                if marker_id == target_id:
                    
                    return corners[i], marker_id, True
    
        # Return None if the target ID is not found
        return None,None,False    
        
    @staticmethod
    def process_marker(corners, marker_id, target_id, camera_matrix, dist_coeffs,marker_centers):
        
        if marker_id != target_id:
            return None, None, None, None
    
        # Get the 2D positions of the marker corners
        marker_2d = corners[0]
    
        # Define the 3D positions of the marker corners in the world frame
        marker_corners_3d = np.array([
            [-0.45, -0.45, 0],
            [0.45, -0.45, 0],
            [0.45, 0.45, 0],
            [-0.45, 0.45, 0]
        ], dtype=np.float32)
    
        # Solve PnP to find rotation and translation vectors
        _, rvec_marker, tvec_marker = cv2.solvePnP(marker_corners_3d, marker_2d, camera_matrix, dist_coeffs)
    
        # Project the 3D points to 2D
        projected_points, _ = cv2.projectPoints(marker_corners_3d, rvec_marker, tvec_marker, camera_matrix, dist_coeffs)
        projected_points = projected_points.reshape(-1, 2).astype(int)
    
        # Calculate the center of the marker
        center = tuple(np.mean(projected_points, axis=0).astype(int))
        marker_centers.append(center)
        
        return center, projected_points, tvec_marker, rvec_marker, marker_centers,marker_corners_3d
        
    @staticmethod
    def resize_image(image, new_width=60):
    
        # Get the original dimensions of the image
        height, width = image.shape[:2]
        
        # Calculate the new height to maintain the aspect ratio
        new_height = int((new_width / width) * height)
        
        # Resize the image using nearest neighbor interpolation
        resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_NEAREST)
        
        return resized_image
    

    

# Position Update

In [190]:
class PositionUpdate:
    def __init__(self):
        pass
    
    @staticmethod
    def draw_trajectory(img, odom_position, color=(0, 255, 255), thickness=10):
        for k in range(1, len(odom_position)):
            start_point = tuple(odom_position[k-1][:2])  # First two elements of the previous position
            end_point = tuple(odom_position[k][:2])     # First two elements of the current position
            cv2.line(img, start_point, end_point, color, thickness)
            
    @staticmethod
    def draw_path(image, marker_centers, color=(255, 0, 0), thickness=2):
        if len(marker_centers) > 1:
            for j in range(1, len(marker_centers)):
                cv2.line(image, tuple(marker_centers[j-1]), tuple(marker_centers[j]), color, thickness)    

    @staticmethod
    def update_odom_position(odom_tvec, marker_corners_3d, newcameramtx, dist, period,centers,yaw,initial,aruco_detected,U,Z):
        
        odom_rvec=  np.array([[0.], [0.], [0.]])
        X_updated, P_updated, X_predicted = kf.kalman_filter(aruco_detected, U, Z, period)
        X_updated=np.round(X_predicted,5)
        yaw=X_updated[2]
        odom_tvec=np.array([[X_updated[0]],[X_updated[1]],np.array([ initial])], dtype=np.float64).reshape(3, 1)
        
        # Convert odometry vectors to the appropriate format
        odom_tvec = np.array(odom_tvec, dtype=np.float64).reshape(3, 1)
        odom_rvec = np.array(odom_rvec, dtype=np.float64).reshape(3, 1)
        projected_points_odom, _ = cv2.projectPoints(marker_corners_3d,odom_rvec,odom_tvec,newcameramtx, dist)
        projected_points_odom = projected_points_odom.reshape(-1, 2).astype(int)
    
        # Compute the center of the projected points
        center_odom = np.mean(projected_points_odom, axis=0).astype(int)
        centers.append(center_odom)
        return odom_tvec, odom_rvec, center_odom, period,centers,yaw

    @staticmethod
    def euler_from_rvec(rvec):
        R, _ = cv2.Rodrigues(rvec)
        
        # Calculate Euler angles (roll, pitch, yaw)
        pitch = np.arctan2(R[2, 0], np.sqrt(R[0, 0]**2 + R[1, 0]**2))
        roll = np.arctan2(R[2, 1], R[2, 2])
        yaw = np.degrees(np.arctan2(R[1, 0], R[0, 0]))
        if yaw >-180 and yaw < 0:
            yaw=abs(yaw)
        else:
            yaw=360-yaw
            
        return roll, pitch, yaw
     


# Main

In [235]:
camera, aruco_dict, aruco_params, newcameramtx, dist, mtx =Image_EKF.initialize_camera_and_calibration()

In [239]:
# Import tdmclient Notebook environment:
import tdmclient.notebook

await tdmclient.notebook.start()

In [350]:

motor_left_target=-200
motor_right_target=-200


In [355]:

motor_left_target=20
motor_right_target=20


In [356]:

odom_position=[]
first=False
current_time=time.time()
# Processing loop
period=0
yaw_odom=0
yaw_aruco=0
marker_centers=[]
odom_centers=[]
initial=False
kidnapped=False
odom_tvec=np.array([[0.],[0.],[0.]])
odom_rvec=np.array([[0.],[0.],[0.]])
tvec_marker=np.array([[0.],[0.],[0.]])
initial_tz=0
kf = KalmanFilter()
marker_corners_3d=0

#kf.U=[20,20]
while True:
    current_time=time.time()
    ret, img = camera.read()
    if not ret:
        print("Error: Could not read from the camera.")
        break

    # Convert to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Undistort the image
    undistorted = cv2.undistort(img_gray, mtx, dist, None, newcameramtx)
    corners,marker_id, robot_detected = Image_EKF.detect_marker_with_id(undistorted, aruco_dict, aruco_params, target_id=5)
    if robot_detected:
        
        cv2.aruco.drawDetectedMarkers(img, [corners],np.array([marker_id]), (0, 255, 0))
        center, projected_points, tvec_marker, rvec_marker ,marker_centers,marker_corners_3d= Image_EKF.process_marker(corners, marker_id, 5, newcameramtx, dist,marker_centers)
        kf.Z[0]=tvec_marker[0][0]
        kf.Z[1]=tvec_marker[1][0]
        kf.Z[2] = np.radians(PositionUpdate.euler_from_rvec(rvec_marker)[2])
        
        if not initial or kidnapped :
            
            odom_tvec = tvec_marker
            odom_centers.append(center)
            initial_tz=odom_tvec[2][0]
            kf.X=kf.Z
            #kf.X[2]=0
            kf.U=speeds()
            odom_tvec, odom_rvec, center_odom, period,odom_centers,yaw_odom = PositionUpdate.update_odom_position(odom_tvec, marker_corners_3d, newcameramtx, dist, period,odom_centers,0,initial_tz,False,kf.U,kf.Z)
            initial=True
            kf.theta=0
            end_time = time.time()
            period = end_time - current_time
    else:
        print("robot not found.")
        kf.U=speeds()
        #current_time=time.time()
        dom_tvec, odom_rvec, center_odom, period,odom_centers ,yaw_odom= PositionUpdate.update_odom_position(odom_tvec, marker_corners_3d, newcameramtx, dist, period,odom_centers,yaw_odom,initial_tz,True,kf.U,kf.Z)
        kf.theta=yaw_odom
        end_time = time.time()
        period = end_time - current_time
    if initial:
        pass
        kf.U=speeds()
        #current_time=time.time()
        dom_tvec, odom_rvec, center_odom, period,odom_centers ,yaw_odom= PositionUpdate.update_odom_position(odom_tvec, marker_corners_3d, newcameramtx, dist, period,odom_centers,yaw_odom,initial_tz,True,kf.U,kf.Z)
        kf.theta=yaw_odom
        end_time = time.time()
        period = end_time - current_time
            
 
    #PositionUpdate.draw_trajectory(img,odom_centers)
 
    cv2.putText(img, "yaw aruco"+ str(np.round(np.degrees((kf.Z[2])),3)), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(img, "yaw"+str(np.round(np.degrees(kf.theta))), (400, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    PositionUpdate.draw_path(img,odom_centers,(255,255,0),10)
    PositionUpdate.draw_path(img, marker_centers,(255,0,0),5)
    
    cv2.imshow("ArUco Detection", img)
    print(kf.X)
    print(period)


    '''
    warped=warp_and_threshold(img,rect_corners)
   
    for i in range(1, len(path)):
        cv2.line(warped, tuple(path[i - 1]), tuple(path[i]), (255, 0, 0), 1)

    
    matplotlib_size = (800, 600)
    opencv_size = (800, 600)
    
    # Generate the imagp.de= map_anddraw_path(warped,pa opencv_size)
    cv2.imshow("warped")arped )
    '''
    if cv2.waitKey(1) & 0xFF == ord('q'):
        motors(0, 0, verbose=True)
        break
    time.sleep(0.02)
#camera.release()
cv2.destroyAllWindows()


[-19.33617048 -11.49848421   6.15888366]
0.03834414482116699
[-19.46101354 -11.58098131   6.19059883]
0.01745319366455078
[-19.47537918 -11.59446814   6.19248589]
0.017791271209716797
[-19.49991018 -11.61486282   6.20923038]
0.024314403533935547
[-19.51123666 -11.62691808   6.21887117]
0.016052961349487305
[-19.5198781  -11.63508594   6.22076789]
0.016462326049804688
[-19.48465494 -11.61492779   6.17610907]
0.017117023468017578
[-19.49129492 -11.62062295   6.1546885 ]
0.014953136444091797
[-19.49118196 -11.62218912   6.14121247]
0.014959335327148438
[-19.49056748 -11.62330884   6.13341284]
0.015596389770507812
[-19.48938329 -11.62408758   6.12874521]
0.018075942993164062
[-19.46595863 -11.61238366   6.12974634]
0.016972064971923828
[-19.44603895 -11.60238699   6.11243538]
0.0158689022064209
[-19.42891158 -11.59371521   6.1037173 ]
0.017514467239379883
[-19.43063446 -11.59705331   6.09792107]
0.014791011810302734
[-19.43028454 -11.59867847   6.09554565]
0.015494108200073242
[-19.4294246